In [2]:
# groupby("user").size()

import torch
import lenskit
import pandas as pd
import pickle
import lenskit.crossfold as xf
import numpy as np

In [10]:
ratings = pd.read_csv("../data/ml-1m/ratings.dat", sep="::", engine="python", header=None)

In [13]:
ratings = ratings.rename(columns={0: "user", 1: "item_id", 2: "rating", 3: "timestamp"})

In [21]:
#ratings = pd.read_csv("../data/ml-20m/ratings.csv")
#ratings = ratings.rename(columns={"userId": "user", "movieId": "item_id"})
#ratings.groupby("item_id").size()
ratings

,user,item_id,rating,timestamp,item
0,1,1193,5,978300760,1104
1,1,661,3,978302109,639
2,1,914,3,978301968,853
3,1,3408,4,978300275,3177
4,1,2355,5,978824291,2162
...,...,...,...,...,...
1000204,6040,1091,1,956716541,1019
1000205,6040,1094,5,956704887,1022
1000206,6040,562,5,956704746,548
1000207,6040,1096,4,956715648,1024


In [22]:
item_map = ratings.groupby("item_id").size().reset_index()

In [23]:
keys = item_map["item_id"].values
values = np.arange(item_map.shape[0])
item_dict = dict([(k, v) for (k, v) in zip(keys, values)])

In [24]:
item_indices = [item_dict[item_id] for item_id in ratings["item_id"].values]

In [25]:
ratings["item"] = item_indices

In [26]:
column_titles = ["user", "item", "item_id", "rating", "timestamp"]
ratings = ratings.reindex(columns=column_titles)
ratings["user"] -= 1

In [27]:
ratings

,user,item,item_id,rating,timestamp
0,0,1104,1193,5,978300760
1,0,639,661,3,978302109
2,0,853,914,3,978301968
3,0,3177,3408,4,978300275
4,0,2162,2355,5,978824291
...,...,...,...,...,...
1000204,6039,1019,1091,1,956716541
1000205,6039,1022,1094,5,956704887
1000206,6039,548,562,5,956704746
1000207,6039,1024,1096,4,956715648


In [28]:
split1 = xf.partition_users(ratings, 1, xf.SampleFrac(0.3))
train_valtest = next(split1)
train = train_valtest.train
valtest = train_valtest.test

In [29]:
split2 = xf.partition_users(valtest, 1, xf.SampleFrac(0.5))
val_test = next(split2)
val = val_test.train
test = val_test.test

In [30]:
train.groupby("user").size()

user
0        37
1        90
2        36
3        15
4       139
       ... 
6035    622
6036    141
6037     14
6038     86
6039    239
Length: 6040, dtype: int64

In [31]:
val.groupby("user").size()

user
0         8
1        19
2         7
3         3
4        29
       ... 
6035    133
6036     31
6037      3
6038     19
6039     51
Length: 6040, dtype: int64

In [32]:
test.groupby("user").size()

user
0         8
1        20
2         8
3         3
4        30
       ... 
6035    133
6036     30
6037      3
6038     18
6039     51
Length: 6040, dtype: int64

In [33]:
train.head()

,user,item,item_id,rating,timestamp
1,0,639,661,3,978302109
3,0,3177,3408,4,978300275
4,0,2162,2355,5,978824291
6,0,1195,1287,5,978302039
7,0,2599,2804,5,978300719


In [34]:
val.head()

,user,item,item_id,rating,timestamp
2,0,853,914,3,978301968
11,0,877,938,4,978301752
18,0,2889,3105,5,978301713
19,0,2592,2797,4,978302039
35,0,740,783,4,978824291


In [35]:
test.head()

,user,item,item_id,rating,timestamp
5,0,1107,1197,3,978302268
20,0,2128,2321,3,978302205
0,0,1104,1193,5,978300760
45,0,963,1028,5,978301777
14,0,970,1035,5,978301753


In [36]:
train.to_pickle("../data/ml-1m-split/train.pkl")
val.to_pickle("../data/ml-1m-split/val.pkl")
test.to_pickle("../data/ml-1m-split/test.pkl")

In [37]:
ratings.to_pickle("../data/ml-1m-split/full.pkl")